In [7]:
"Check data"
import numpy as np 

data = np.load("C:\\Users\\msi\\Desktop\\Constanze\\Docs\\DATA\\PREPROCESSED\\S01\\data.npy")
print(data.shape)
nan_count = np.isnan(data).sum()

# Print the result
print(f"Number of NaN values in the array: {nan_count}")

(461, 64, 7196)
Number of NaN values in the array: 0
